# **Pipeline RAG com LangChain - embeddings → retriever → LLM**

In [1]:
# Notebook: Pipeline RAG com LangChain - embeddings → retriever → LLM

# Objetivo:
# Demonstrar um pipeline básico de RAG utilizando LangChain, onde:
# 1. Criamos embeddings de um conjunto de documentos textuais.
# 2. Construímos um retriever para buscar informações relevantes.
# 3. Utilizamos um LLM para gerar respostas contextuais a partir do conteúdo recuperado.

# IMPORTANTE:
# - Para rodar este notebook você precisa ter:
#   * Python 3.8+
#   * LangChain instalado (`pip install langchain`)
#   * OpenAI SDK instalado (`pip install openai`)
#   * Ter uma chave de API OpenAI configurada na variável ambiente OPENAI_API_KEY
# - Este é um exemplo simplificado para entendimento do pipeline.

# **1. IMPORTAÇÕES E INSTALAÇÕES**

In [2]:
!pip install -q faiss-cpu sentence-transformers langchain langchain-community

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import os

# **2. CRIANDO UM CONJUNTO DE DOCUMENTOS DE EXEMPLO**

In [3]:
docs = [
    "O churn é o cancelamento ou abandono de clientes em um serviço ou produto.",
    "NPS, ou Net Promoter Score, mede a lealdade dos clientes através da pergunta: 'Você recomendaria nossa empresa a um amigo?'",
    "LangChain é uma biblioteca para construir aplicações que usam modelos de linguagem large (LLMs) integrados a outras fontes de dados.",
    "RAG, Retrieval-Augmented Generation, conecta modelos de linguagem a bases de conhecimento através de embeddings e mecanismos de busca para melhorar respostas.",
    "Embeddings representam texto em vetores numéricos que capturam significado semântico, possibilitando busca eficiente por similaridade.",
    "O pipeline básico de RAG inclui: criação de embeddings, uso do retriever para buscar documentos relevantes e geração da resposta pelo LLM.",
]

# **3. CONSTRUÇÃO DOS EMBEDDINGS DOS DOCUMENTOS**

In [4]:
print("Criando embeddings dos documentos com OpenAIEmbeddings...")

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Criar vetor store FAISS para indexar e buscar similaridades
vectorstore = FAISS.from_texts(docs, embeddings)
print(f"Documentos indexados: {len(docs)}")

Criando embeddings dos documentos com OpenAIEmbeddings...


/tmp/ipython-input-2911721343.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

Documentos indexados: 6


# **4. CRIAR O RETRIEVER A PARTIR DO VECTORSTORE**

In [5]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2}
)
print("Retriever pronto (k=2).")

Retriever pronto (k=2).


# **5. CONFIGURAR O MODELO LLM**

In [6]:
import os
USE_OPENAI = "OPENAI_API_KEY" in os.environ and len(os.environ["OPENAI_API_KEY"].strip()) > 0

print("Configurando o LLM local (Hugging Face - FLAN-T5 base)...")
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_community.llms import HuggingFacePipeline

model_name = "google/flan-t5-base"  # leve; bom p/ CPU/GPU do Colab
tok = AutoTokenizer.from_pretrained(model_name)
mdl = AutoModelForSeq2SeqLM.from_pretrained(model_name)
gen_pipe = pipeline("text2text-generation", model=mdl, tokenizer=tok, max_new_tokens=256)

llm = HuggingFacePipeline(pipeline=gen_pipe)

Configurando o LLM local (Hugging Face - FLAN-T5 base)...


Device set to use cuda:0
/tmp/ipython-input-425815035.py:13: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=gen_pipe)


# **6. CRIAR A CHAIN DE RAG: COMBINA RETRIEVER E LLM PARA RESPOSTAS**

In [7]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,             # usa o FAISS retriever
    return_source_documents=False    # mude para True se quiser ver as fontes
)
print("RAG chain pronta.")

RAG chain pronta.


# **7. REALIZAR PERGUNTAS E GERAR RESPOSTAS USANDO O PIPELINE RAG**

In [8]:
query_1 = "O que significa churn em análise de clientes?"
query_2 = "Como funciona o pipeline de RAG?"
query_3 = "Explique o que é LangChain."

print("\nPerguntas e respostas no pipeline RAG:\n")
for query in [query_1, query_2, query_3]:
    print(f"Pergunta: {query}")
    resposta = qa_chain.run(query)
    print(f"Resposta: {resposta}\n{'-'*60}")


/tmp/ipython-input-602582006.py:11: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  resposta = qa_chain.run(query)



Perguntas e respostas no pipeline RAG:

Pergunta: O que significa churn em análise de clientes?
Resposta: NPS, or Net Promoter Score, mede a lealdade dos clientes através da pergunta: 'Você recomendaria nossa empresa a um amigo?'
------------------------------------------------------------
Pergunta: Como funciona o pipeline de RAG?
Resposta: criaço de embeddings, uso do retriever para buscar documents relevantes e geraço da resposta pelo LLM
------------------------------------------------------------
Pergunta: Explique o que é LangChain.
Resposta: a biblioteca para construir aplicaçes que usam modelos de linguagem large (LLMs) integrados a outras fontes de dados
------------------------------------------------------------


# **8. CONCLUSÃO**

In [9]:
print("""
Neste notebook você viu como:
- Criar embeddings semânticos para documentos usando OpenAIEmbeddings.
- Indexar documentos em um vetor store FAISS para busca eficiente.
- Usar o retriever para buscar os documentos mais relevantes para uma query.
- Passar o conteúdo recuperado para um LLM gerar respostas contextuais e contextualizadas.
Este é o fluxo típico da pipeline RAG com LangChain: embeddings → retriever → LLM.
""")


Neste notebook você viu como:
- Criar embeddings semânticos para documentos usando OpenAIEmbeddings.
- Indexar documentos em um vetor store FAISS para busca eficiente.
- Usar o retriever para buscar os documentos mais relevantes para uma query.
- Passar o conteúdo recuperado para um LLM gerar respostas contextuais e contextualizadas.
Este é o fluxo típico da pipeline RAG com LangChain: embeddings → retriever → LLM.

